# <font color=red> Tutorial 4 - Pandas Advanced</font>

In [1]:
import pandas as pd
import numpy as np

## Reshaping a Dataframe Using The Melt Method

In [2]:
temperature_data_dict = {
    'Day' : ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday'],
    'Haifa':     [30, 29, 28, 29, 30, 31, 32],
    'Tel-Aviv':  [28, 27, 27, 28, 29, 30, 31],
    'Beersheva': [32, 32, 31, 31, 31, 33, 34]
}

In [3]:
df = pd.DataFrame(temperature_data_dict)
df

,Day,Haifa,Tel-Aviv,Beersheva
0,Sunday,30,28,32
1,Monday,29,27,32
2,Tuesday,28,27,31
3,Wednesday,29,28,31
4,Thursday,30,29,31
5,Friday,31,30,33
6,Saturday,32,31,34


For data analysis purposes we would like to transform this dataframe so we will have an entire column for temperature to be able to use aggregating methods such as mean, max, min, etc.

We can use the melt() method to transform a DataFrame from wide format to long format. The `id_vars` parameter gets the columns we don't want to change. The `var_name` parameter gets the name of the column that includes all other columns besides the columns defined using the `id_vars`. In our case we will create a new column named 'City' to include all other columns - Haifa, Tel-Aviv, and Beersheva. The `value_name` parameter is the name of the column that will contain all values, in our case it's the temperatures, so we will name it "Temperature".

In [4]:
melted_df = pd.melt(df, id_vars='Day', var_name='City', value_name='Temperature')
melted_df

,Day,City,Temperature
0,Sunday,Haifa,30
1,Monday,Haifa,29
2,Tuesday,Haifa,28
3,Wednesday,Haifa,29
4,Thursday,Haifa,30
5,Friday,Haifa,31
6,Saturday,Haifa,32
7,Sunday,Tel-Aviv,28
8,Monday,Tel-Aviv,27
9,Tuesday,Tel-Aviv,27


## Merging Multiple Dataframes

Let's assume that we got a data that was splitted into 3 different files. We would like to assemble them into one file that contains all data.

In [5]:
raw_data = {
        'subject_id': ['1', '2', '3', '4', '5'],
        'first_name': ['Alex', 'Amy', 'Allen', 'Alice', 'Ayoung'], 
        'last_name': ['Anderson', 'Ackerman', 'Ali', 'Aoni', 'Atiches']}
df_a = pd.DataFrame(raw_data)
df_a

,subject_id,first_name,last_name
0,1,Alex,Anderson
1,2,Amy,Ackerman
2,3,Allen,Ali
3,4,Alice,Aoni
4,5,Ayoung,Atiches


In [6]:
raw_data = {
        'subject_id': ['6', '7', '8', '9', '10'],
        'first_name': ['Billy', 'Brian', 'Bran', 'Bryce', 'Betty'], 
        'last_name': ['Bonder', 'Black', 'Balwner', 'Brice', 'Btisan']}
df_b = pd.DataFrame(raw_data)
df_b

,subject_id,first_name,last_name
0,6,Billy,Bonder
1,7,Brian,Black
2,8,Bran,Balwner
3,9,Bryce,Brice
4,10,Betty,Btisan


In [7]:
raw_data = {
        'subject_id': ['5', '2', '7', '4', '1', '3', '10', '9', '8'],
        'test_id': [51, 15, 15, 61, 16, 14, 15, 1, 61]}
df_test_id = pd.DataFrame(raw_data)
df_test_id

,subject_id,test_id
0,5,51
1,2,15
2,7,15
3,4,61
4,1,16
5,3,14
6,10,15
7,9,1
8,8,61


In [8]:
all_rows_df = pd.concat([df_a, df_b], ignore_index=True)
all_rows_df

,subject_id,first_name,last_name
0,1,Alex,Anderson
1,2,Amy,Ackerman
2,3,Allen,Ali
3,4,Alice,Aoni
4,5,Ayoung,Atiches
5,6,Billy,Bonder
6,7,Brian,Black
7,8,Bran,Balwner
8,9,Bryce,Brice
9,10,Betty,Btisan


Notice that the `'subject_id'` column in `df_test_id` is not sorted as in the `all_rows_df`. The `merge()` method in Pandas can handle this, we just have to provide the column name on which we want to merge using the `on` parameter. In our case it's the `'subject_id'` column on which we want to merge the dataframes.

In [9]:
merged_df = pd.merge(all_rows_df, df_test_id, on='subject_id')
merged_df

,subject_id,first_name,last_name,test_id
0,1,Alex,Anderson,16
1,2,Amy,Ackerman,15
2,3,Allen,Ali,14
3,4,Alice,Aoni,61
4,5,Ayoung,Atiches,51
5,7,Brian,Black,15
6,8,Bran,Balwner,61
7,9,Bryce,Brice,1
8,10,Betty,Btisan,15


Did you noticed anything strange?


When merging the dfs by `'subject_id'`, the observation with subject_id = 6, which was missing from ``df_test_id``, was deleted due to the merge.

To preserve this observation we must add how='outer' argument to pd.merge

In [10]:
merged_df2 = pd.merge(all_rows_df, df_test_id, on='subject_id', how='outer')
merged_df2

,subject_id,first_name,last_name,test_id
0,1,Alex,Anderson,16.0
1,2,Amy,Ackerman,15.0
2,3,Allen,Ali,14.0
3,4,Alice,Aoni,61.0
4,5,Ayoung,Atiches,51.0
5,6,Billy,Bonder,NaN
6,7,Brian,Black,15.0
7,8,Bran,Balwner,61.0
8,9,Bryce,Brice,1.0
9,10,Betty,Btisan,15.0


## Group By (Split-Apply-Combine)

The `DataFrame.groupby()` method splits the data into different groups depending on a variable of your choice. We use the `groupby()` method to apply some functions on each group. Let's experiment on the following example:

### Read Data from CSV file

We will use the attached CSV file named `'weather_by_cities.csv'`. 

In [11]:
df = pd.read_csv("weather_by_cities.csv")
df

,day,city,temperature,windspeed,event
0,1/1/2017,new york,32,6,Rain
1,1/2/2017,new york,36,7,Sunny
2,1/3/2017,new york,28,12,Snow
3,1/4/2017,new york,33,7,Sunny
4,1/1/2017,mumbai,90,5,Sunny
5,1/2/2017,mumbai,85,12,Fog
6,1/3/2017,mumbai,87,15,Fog
7,1/4/2017,mumbai,92,5,Rain
8,1/1/2017,paris,45,20,Sunny
9,1/2/2017,paris,50,13,Cloudy


Now let's use the `groupby()` method to try and answer the following questions:
1. What is the maximum temperature in each of the cities?
2. What is the average wind speed in each of the cities?

### Group By (Split-Apply-Combine) - Split

In [12]:
gb_city = df.groupby('city')
gb_city

We can see that we got a groupby object. Here is a visual representation of how this object looks like:

<img src='group_by_cities.png'></img>

A `groupby` object is a key-value object. In our example, the key is the city (e.g. 'new york'), and the value is a `Dataframe` object that contains the city's data.

#### Accessing the Groups

In [13]:
for city, city_df in gb_city:
    print(city)
    print(city_df)

mumbai
        day    city  temperature  windspeed  event
4  1/1/2017  mumbai           90          5  Sunny
5  1/2/2017  mumbai           85         12    Fog
6  1/3/2017  mumbai           87         15    Fog
7  1/4/2017  mumbai           92          5   Rain
new york
        day      city  temperature  windspeed  event
0  1/1/2017  new york           32          6   Rain
1  1/2/2017  new york           36          7  Sunny
2  1/3/2017  new york           28         12   Snow
3  1/4/2017  new york           33          7  Sunny
paris
         day   city  temperature  windspeed   event
8   1/1/2017  paris           45         20   Sunny
9   1/2/2017  paris           50         13  Cloudy
10  1/3/2017  paris           54          8  Cloudy
11  1/4/2017  paris           42         10  Cloudy


We can access a specific group using the `get_group()` method:

In [14]:
gb_city.get_group('mumbai')

,day,city,temperature,windspeed,event
4,1/1/2017,mumbai,90,5,Sunny
5,1/2/2017,mumbai,85,12,Fog
6,1/3/2017,mumbai,87,15,Fog
7,1/4/2017,mumbai,92,5,Rain


### Group By (Split-Apply-Combine) - Apply & Combine

After we split the data, we can apply a method (e.g. max, mean) and get the results combined into a single Dataframe:

In [15]:
gb_city.max()

,day,temperature,windspeed,event
city,,,,
mumbai,1/4/2017,92,15,Sunny
new york,1/4/2017,36,12,Sunny
paris,1/4/2017,54,20,Sunny


### Side note about warnings

Warning messages are typically issued in situations where it is useful to alert the user of some condition in a program, where that condition (normally) doesn’t warrant raising an exception and terminating the program. For example, 'FutureWarning' are intended for end users of applications that are written in Python.

In [16]:
gb_city.mean()

C:\Users\sapir\AppData\Local\Temp\ipykernel_28056\2245304035.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  gb_city.mean()


,temperature,windspeed
city,,
mumbai,88.50,9.25
new york,32.25,8.00
paris,47.75,12.75


<img src='split_apply_combine.png'></img>

We can apply various methods on any column we choose on a `groupby` object using the `agg` method, we just have to provide a tuple with the column name and the method we want to use. The result we get is a single Dataframe that contains all the data we are interested in:

In [17]:
gb_city.agg(mean_temp = ('temperature', 'mean'),
            std_temp = ('temperature', 'std'),
            max_wind = ('windspeed', 'max'),
            min_wind = ('windspeed', 'min')
           )

,mean_temp,std_temp,max_wind,min_wind
city,,,,
mumbai,88.50,3.109126,15,5
new york,32.25,3.304038,12,6
paris,47.75,5.315073,20,8


Use the auto-complete (TAB) to explore more possibilities:

In [18]:
#gb_city.

We can use the `apply()` method to run other types of methods on each group:

In [19]:
gb_city.apply(len)

city
mumbai      4
new york    4
paris       4
dtype: int64

We can also define custom functions:

In [20]:
def wind_temp_ratio(df):
    return sum(df['windspeed']) / sum(df['temperature'])

In [21]:
gb_city.apply(wind_temp_ratio)

city
mumbai      0.104520
new york    0.248062
paris       0.267016
dtype: float64

## Exploratory Data Analysis (EDA)

After we explored various Pandas features, let's use them to explore a real data-set. We are going to explore a movies data-set that contains 3 different files - users data file, users' ratings data file, and movies' data file.

It is a good practice to explore the data files before loading them (using any text editor) and check:
1. Does the file includes an index column? 
2. Does the file includes a first row of column names or we have to create them manually?
3. Does the file uses the default separator (',')?

We will use the read_csv method to load data from a file. By default, the method expects the column separator to be a comma, but we can change that using the 'sep' parameter in cases in which there is a different separator (like in our case). <br>
The columns names should be given in the first line of the file, but sometimes they are not provided (like in our case) and we should insert them as a list using the 'names=' parameter.

In [22]:
users_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('u.user', sep='|', names=users_cols)

ratings_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv('u.data', sep='\t', names=ratings_cols)

movies_cols = ['movie_id', 'title', 'release_date', 'video_release_date', 'imdb_url', 'genre']
movies = pd.read_csv('u.item.new', sep='|', names=movies_cols)

### Initial Data Exploration

Let's explore the data:

In [23]:
users.shape

(943, 5)

In [24]:
users.head()

,user_id,age,sex,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [25]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 943 entries, 0 to 942
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     943 non-null    int64 
 1   age         943 non-null    int64 
 2   sex         943 non-null    object
 3   occupation  943 non-null    object
 4   zip_code    943 non-null    object
dtypes: int64(2), object(3)
memory usage: 37.0+ KB


We can use the `describe()` method to get basic information regarding the numerical column types in the dataframe, we can use the `include='all'` parameter to provide info of all the columns and not only the numerical types:

In [26]:
users.describe()

,user_id,age
count,943.000000,943.000000
mean,472.000000,34.051962
std,272.364951,12.192740
min,1.000000,7.000000
25%,236.500000,25.000000
50%,472.000000,31.000000
75%,707.500000,43.000000
max,943.000000,73.000000


In [27]:
users.describe(include='all')

,user_id,age,sex,occupation,zip_code
count,943.000000,943.000000,943,943,943
unique,NaN,NaN,2,21,795
top,NaN,NaN,M,student,55414
freq,NaN,NaN,670,196,9
mean,472.000000,34.051962,NaN,NaN,NaN
std,272.364951,12.192740,NaN,NaN,NaN
min,1.000000,7.000000,NaN,NaN,NaN
25%,236.500000,25.000000,NaN,NaN,NaN
50%,472.000000,31.000000,NaN,NaN,NaN
75%,707.500000,43.000000,NaN,NaN,NaN


In [28]:
ratings.shape

(100000, 4)

In [29]:
ratings.head()

,user_id,movie_id,rating,unix_timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [30]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 4 columns):
 #   Column          Non-Null Count   Dtype
---  ------          --------------   -----
 0   user_id         100000 non-null  int64
 1   movie_id        100000 non-null  int64
 2   rating          100000 non-null  int64
 3   unix_timestamp  100000 non-null  int64
dtypes: int64(4)
memory usage: 3.1 MB


In [31]:
ratings.describe(include='all')

,user_id,movie_id,rating,unix_timestamp
count,100000.00000,100000.000000,100000.000000,1.000000e+05
mean,462.48475,425.530130,3.529860,8.835289e+08
std,266.61442,330.798356,1.125674,5.343856e+06
min,1.00000,1.000000,1.000000,8.747247e+08
25%,254.00000,175.000000,3.000000,8.794487e+08
50%,447.00000,322.000000,4.000000,8.828269e+08
75%,682.00000,631.000000,4.000000,8.882600e+08
max,943.00000,1682.000000,5.000000,8.932866e+08


In [32]:
movies.shape

(1682, 6)

In [33]:
movies.head()

,movie_id,title,release_date,video_release_date,imdb_url,genre
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,Comedy
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,Thriller
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,Thriller
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,Drama
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),Thriller


In [34]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1682 entries, 0 to 1681
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   movie_id            1682 non-null   int64  
 1   title               1682 non-null   object 
 2   release_date        1681 non-null   object 
 3   video_release_date  0 non-null      float64
 4   imdb_url            1679 non-null   object 
 5   genre               1682 non-null   object 
dtypes: float64(1), int64(1), object(4)
memory usage: 79.0+ KB


In [35]:
movies.describe(include='all')

,movie_id,title,release_date,video_release_date,imdb_url,genre
count,1682.000000,1682,1681,0.0,1679,1682
unique,NaN,1664,240,NaN,1660,19
top,NaN,"Designated Mourner, The (1997)",01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Designated%20...,Drama
freq,NaN,2,215,NaN,2,505
mean,841.500000,NaN,NaN,NaN,NaN,NaN
std,485.695893,NaN,NaN,NaN,NaN,NaN
min,1.000000,NaN,NaN,NaN,NaN,NaN
25%,421.250000,NaN,NaN,NaN,NaN,NaN
50%,841.500000,NaN,NaN,NaN,NaN,NaN
75%,1261.750000,NaN,NaN,NaN,NaN,NaN


Let's check how many users rated the movies, we can use the `nunique()` method:

In [36]:
ratings.user_id.nunique()

943

### Cleaning the Data

Almost every dataset is likely to have some missing data. In our case, the movies data contained some missing values that we will have to take care of before working with the data.

First, let's see what exactly are missing values using the Pandas `isna()` method:

In [37]:
pd.isna('NaN')

False

In [38]:
pd.isna('Missing')

False

In [39]:
pd.isna('NA')

False

In [40]:
pd.isna(None)

True

In [41]:
pd.isna(np.nan)

True

We can see that `None` types or `np.nan` types are recognized as missing values, but strings such as 'NaN' and 'NA' are not.

we can use the `count()` method to count the number of non-NA values for each column, or alternatively use the `isna()` method on our Dataframe:

In [42]:
movies.count()

movie_id              1682
title                 1682
release_date          1681
video_release_date       0
imdb_url              1679
genre                 1682
dtype: int64

In [43]:
movies.isna()

,movie_id,title,release_date,video_release_date,imdb_url,genre
0,False,False,False,True,False,False
1,False,False,False,True,False,False
2,False,False,False,True,False,False
3,False,False,False,True,False,False
4,False,False,False,True,False,False
...,...,...,...,...,...,...
1677,False,False,False,True,False,False
1678,False,False,False,True,False,False
1679,False,False,False,True,False,False
1680,False,False,False,True,False,False


Now let's use the `sum()` method to get the overall number of missing values for each column:

In [44]:
movies.isna().sum()

movie_id                 0
title                    0
release_date             1
video_release_date    1682
imdb_url                 3
genre                    0
dtype: int64

In [45]:
movies[movies['release_date'].isna() == True]

,movie_id,title,release_date,video_release_date,imdb_url,genre
266,267,unknown,NaN,NaN,NaN,unknown


In this example the 'unknown' title will not be recognized as a missing value. If we want we can use the `replace()` method to adjust these cases:

In [46]:
pd.isna(movies.loc[266, 'title'])

False

In [47]:
movies.replace('unknown', np.nan, inplace=True)
movies[movies['release_date'].isna() == True]

,movie_id,title,release_date,video_release_date,imdb_url,genre
266,267,NaN,NaN,NaN,NaN,NaN


In [48]:
movies[movies['imdb_url'].isna() == True]

,movie_id,title,release_date,video_release_date,imdb_url,genre
266,267,NaN,NaN,NaN,NaN,NaN
1357,1358,The Deadly Cure (1996),16-Sep-1996,NaN,NaN,Action
1358,1359,Boys in Venice (1996),24-Sep-1996,NaN,NaN,Drama


The `dropna()` method drops all rows in which we have a missing value, in our case it's risky, since the entire `'video_release_date'` has missing values, so if we will use the `dropna()` method it will drop the entire dataset. To avoid this we can specify the columns on which we want to use the `dropna()` method using the `subset` parameter:

In [49]:
movies_cleaned = movies.dropna(subset=['release_date', 'imdb_url'])
movies_cleaned.shape

(1679, 6)

In [50]:
movies_final = movies_cleaned.drop(columns=['video_release_date'])
movies_final.shape

(1679, 5)

In [51]:
movies_final.head()

,movie_id,title,release_date,imdb_url,genre
0,1,Toy Story (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Toy%20Story%2...,Comedy
1,2,GoldenEye (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?GoldenEye%20(...,Thriller
2,3,Four Rooms (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Four%20Rooms%...,Thriller
3,4,Get Shorty (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Get%20Shorty%...,Drama
4,5,Copycat (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Copycat%20(1995),Thriller


## <font color=blue> **Exercise** </font>
1. Present the users distribution by gender
1. What are the main 5 occupations of users?
2. What are the 5 most rated movies (5 movies with the highest number of ratings)? (**Hint**: explore the data, does the movies' Dataframe contains any ratings information?)
3. What are the top 5 rated movies (5 movies with the highest average rating)? <br>
Are the movies in the results reliable? Check how many times those movies were rated. (**Hint**: use agg() to run both size and mean methods)
4. Repeat question 4, now count only movies that rated at least 200 times.

In [52]:
# 1
gb_gender_obj = users.groupby('sex')
gb_gender_obj.size()

# 2
users_gb_occupation = users.groupby('occupation')
users_gb_occupation.size().sort_values(ascending=False).head()

# 3
movies.head()

ratings.head()

movies_ratings_merged = pd.merge(movies_final, ratings, on='movie_id')
movies_ratings_merged.head()

movies_gb_title = movies_ratings_merged.groupby('title')

movies_gb_title.size().sort_values(ascending=False).head()

# 4
movies_size_mean = movies_gb_title['rating'].agg(ratings_count='size', ratings_mean='mean')
movies_size_mean.sort_values(by='ratings_mean', ascending=False).head()

# alternative
# movies_size_mean = movies_gb_title.agg(ratings_count=('rating', 'size'), ratings_mean=('rating', 'mean') )
# movies_size_mean.sort_values(by='ratings_mean', ascending=False).head()

# 5
filt = movies_size_mean['ratings_count'] > 200
movies_top_5 = movies_size_mean.loc[filt]
movies_top_5.sort_values(by='ratings_mean', ascending=False).head()